Computing statistics for online data can result in considerable loss of accuracy.  Here we experiment with the Welford algorithm that gives good results for the standard deviation.

See [Wikipedia](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_online_algorithm) for a description of the algorithm.

# Requirements

In [53]:
import decimal as dc
import math
import numpy as np
import random

# Data set

We create a data set consisting of two parts, the first contains number an order of magnitude larger than the second.

In [22]:
data_large = np.arange(1_000_000.0, 1_000_000_000_000.0, 100_000.0)

In [23]:
data_large.sum()

4.9999994999955e+18

In [24]:
data_small = np.arange(0.001, 100.0, 0.01)

In [25]:
data_small.sum()

499959.99999999994

The data sets are combined in two ways, the small number first, followed by the larger numbers, and vice versa.

In [26]:
small_first_data = np.concatenate([data_small, data_large])

In [27]:
small_first_data.sum()

4.999999499995999e+18

In [28]:
large_first_data = np.concatenate([data_large, data_small])

In [34]:
for _ in range(5):
    np.random.shuffle(large_first_data)
    print(large_first_data.sum())

4.999999499995992e+18
4.999999499995991e+18
4.999999499996007e+18
4.999999499995999e+18
4.999999499996e+18


# Algorithms

For convenience, all the algorithms are implemented in classes with a uniform interface.

The first algorithm is the Welford algorithm.

In [39]:
class WelfordStats:
    
    def __init__(self):
        self._n = 0
        self._avg = 0.0
        self._m2 = 0.0
        
    def add(self, value):
        self._n += 1
        prev_avg = self._avg
        self._avg += (value - self._avg)/self._n
        self._m2 += (value - self._avg)*(value -prev_avg)
        
    @property
    def n(self):
        return self._n
    
    @property
    def mean(self):
        return self._avg
    
    @property
    def stddev(self):
        return math.sqrt((self._m2/(self._n - 1)))
        

The second algorithm is the naive one.

In [42]:
class Stats:
    
    def __init__(self):
        self._n = 0
        self._sum = 0.0
        self._sum2 = 0.0
        
    def add(self, value):
        self._n += 1
        self._sum += value
        self._sum2 += value**2
        
    @property
    def n(self):
        return self._n
    
    @property
    def mean(self):
        return self._sum/self._n
    
    @property
    def stddev(self):
        return math.sqrt((self._sum2 - (self._sum*self._sum)/self._n)/(self._n - 1))
        

The third implementation uses Python's `decimal` module to preserve accuracy.

In [57]:
class DecimalStats:
    
    def __init__(self, precision=50):
        dc.getcontext().prec = precision
        self._n = dc.Decimal(0)
        self._sum = dc.Decimal(0.0)
        self._sum2 = dc.Decimal(0.0)
        
    def add(self, value):
        self._n += dc.Decimal(1)
        self._sum += dc.Decimal(value)
        self._sum2 += dc.Decimal(value**2)
        
    @property
    def n(self):
        return self._n
    
    @property
    def mean(self):
        return self._sum/self._n
    
    @property
    def stddev(self):
        return ((self._sum2 - (self._sum*self._sum)/self._n)/(self._n - 1)).sqrt()
        

In [55]:
stats = Stats()
welford_stats = WelfordStats()
decimal_stats = DecimalStats()
for value in np.arange(1.0, 5.0, 1.0):
    stats.add(value)
    welford_stats.add(value)
    decimal_stats.add(value)
print(stats.mean, stats.stddev)
print(welford_stats.mean, welford_stats.stddev)
print(decimal_stats.mean, decimal_stats.stddev)

2.5 1.2909944487358056
2.5 1.2909944487358056
2.5 1.2909944487358056283930884665941332036109739017639


# Comparison

We run the algorithm on one of the data sets.

In [62]:
stats = Stats()
welford_stats = WelfordStats()
decimal_stats = DecimalStats()
for value in large_first_data:
    stats.add(value)
    welford_stats.add(value)
    decimal_stats.add(value)
print((dc.Decimal(stats.stddev) - decimal_stats.stddev))
print((dc.Decimal(welford_stats.stddev) - decimal_stats.stddev))

0.06717778612280627061929645420703303721
0.00010014940405627061929645420703303721


It is indeed clear that the standard deviation computed using the Welford algorithm is indeed more accurate than that by the naive method.  Note however that the mean value is more accurate by simply summing the data values.

It is not a panacea though.

In [61]:
stats = Stats()
welford_stats = WelfordStats()
decimal_stats = DecimalStats()
for _ in range(1_000_000):
    value = random.gauss(mu=0.0, sigma=10_000.0)
    stats.add(value)
    welford_stats.add(value)
    decimal_stats.add(value)
print((dc.Decimal(stats.stddev) - decimal_stats.stddev))
print((dc.Decimal(welford_stats.stddev) - decimal_stats.stddev))

5.5380491384252865937575867883120891E-11
2.02718633071467240479812196008120891E-10
